In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
import tensorflow_probability as tfp

In [2]:
gdrdata = pd.read_csv('/content/gdr parameters theoretical.csv')
gdrdata

,Z,A,delta,El,Eta,E1[MeV],W1[MeV],E2[MeV],W2[MeV]
0,14,35,0.200000,Si,1.000,21.27,7.76,21.27,7.76
1,14,36,0.222222,Si,1.216,18.63,5.41,22.29,9.55
2,14,37,0.243243,Si,1.310,17.55,4.95,22.51,9.49
3,14,38,0.263158,Si,1.305,17.45,4.81,22.29,9.11
4,14,39,0.282051,Si,1.301,17.32,4.68,22.08,8.87
...,...,...,...,...,...,...,...,...,...
5981,110,335,0.343284,10,1.198,10.31,2.33,12.17,3.41
5982,110,336,0.345238,10,1.198,10.30,2.33,12.15,3.41
5983,110,337,0.347181,10,1.177,10.40,2.35,12.08,3.38
5984,110,338,0.349112,10,1.177,10.38,2.35,12.06,3.37


In [3]:
gdrdata.head(15)

,Z,A,delta,El,Eta,E1[MeV],W1[MeV],E2[MeV],W2[MeV]
0,14,35,0.200000,Si,1.000,21.27,7.76,21.27,7.76
1,14,36,0.222222,Si,1.216,18.63,5.41,22.29,9.55
2,14,37,0.243243,Si,1.310,17.55,4.95,22.51,9.49
3,14,38,0.263158,Si,1.305,17.45,4.81,22.29,9.11
4,14,39,0.282051,Si,1.301,17.32,4.68,22.08,8.87
5,14,40,0.300000,Si,1.364,16.61,4.33,22.12,8.52
6,14,41,0.317073,Si,1.368,16.42,4.13,21.93,7.93
7,14,42,0.333333,Si,1.415,15.89,3.82,21.89,7.34
8,14,43,0.348837,Si,1.323,16.48,4.07,21.34,7.31
9,14,44,0.363636,Si,1.296,16.57,4.13,21.03,7.27


# Rms deviation for theoretical data 

In [4]:
e1data = gdrdata['E1[MeV]'].to_numpy()
e1data

array([21.27, 18.63, 17.55, ..., 10.4 , 10.38,  8.3 ])

In [7]:
sigma_e1 = np.std(e1data)
sigma_e1

2.2000004234459576

In [6]:
e2data = gdrdata['E2[MeV]'].to_numpy()
e2data

array([21.27, 22.29, 22.51, ..., 12.08, 12.06, 11.53])

In [8]:
sigma_e2 = np.std(e2data)
sigma_e2

2.1392005714053113

In [24]:
w1data = gdrdata['W1[MeV]'].to_numpy()
w1data

array([7.76, 5.41, 4.95, ..., 2.35, 2.35, 1.84])

In [25]:
sigma_w1 = np.std(w1data)
sigma_w1

1.0946500160207928

In [26]:
w2data = gdrdata['W2[MeV]'].to_numpy()
w2data

array([7.76, 9.55, 9.49, ..., 3.38, 3.37, 3.25])

In [27]:
sigma_w2 = np.std(w2data)
sigma_w2

1.2628291865343948

# Tunning the data using BNN

In [9]:
ade2 = gdrdata.drop(['Z','El','Eta','E1[MeV]','E2[MeV]','W1[MeV]','W2[MeV]'],axis = 1).to_numpy()
ade2

array([[3.50000000e+01, 2.00000000e-01],
       [3.60000000e+01, 2.22222222e-01],
       [3.70000000e+01, 2.43243243e-01],
       ...,
       [3.37000000e+02, 3.47181009e-01],
       [3.38000000e+02, 3.49112426e-01],
       [3.39000000e+02, 3.51032448e-01]])

In [10]:
y = gdrdata['E2[MeV]'].to_numpy()
y

array([21.27, 22.29, 22.51, ..., 12.08, 12.06, 11.53])

In [11]:
x_train, x_test, y_train, y_test = train_test_split(ade2, y,train_size = 0.85)
print(len(x_train))

5088


#BNN Model development 

In [12]:
hidden_units = [10, 10]
learning_rate = 0.001
num_epochs = 100

def run_experiment(model, loss, x_train,y_train,x_test,y_test):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(x_train,y_train, epochs=num_epochs, validation_data=(x_test,y_test))
    print("Model training finished.")
    _, rmse = model.evaluate(x_train,y_train, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(x_test,y_test, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")


In [13]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model


In [14]:
def create_bnn_model(train_size):
    inputs = layers.Input(shape=(2,), name="input")
    features = layers.BatchNormalization()(inputs)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # The output is a distribution of possible values, so we use the tfp.layers.DistributionLambda layer.
    outputs = tfp.layers.DenseVariational(
        units=1,
        make_prior_fn=prior,
        make_posterior_fn=posterior,
        kl_weight=1 / train_size,
    )(features)
    outputs = tfp.layers.DistributionLambda(lambda x: tfp.distributions.Normal(loc=x, scale=1))(outputs)

    model = keras.Model(inputs=inputs, outputs=outputs)

    return model

In [16]:
num_epochs = 500
mse_loss = keras.losses.MeanSquaredError()
e2_pred_model = create_bnn_model(int(x_train.size ))
run_experiment(e2_pred_model, mse_loss, x_train,y_train,x_test,y_test)

Start training the model...
Epoch 1/500
159/159 [==============================] - 5s 10ms/step - loss: 213.9238 - root_mean_squared_error: 14.6261 - val_loss: 212.8851 - val_root_mean_squared_error: 14.5905
Epoch 2/500
159/159 [==============================] - 1s 3ms/step - loss: 188.0986 - root_mean_squared_error: 13.7148 - val_loss: 171.6131 - val_root_mean_squared_error: 13.1000
Epoch 3/500
159/159 [==============================] - 1s 3ms/step - loss: 156.6690 - root_mean_squared_error: 12.5167 - val_loss: 154.5917 - val_root_mean_squared_error: 12.4334
Epoch 4/500
159/159 [==============================] - 1s 3ms/step - loss: 133.9540 - root_mean_squared_error: 11.5738 - val_loss: 110.6342 - val_root_mean_squared_error: 10.5182
Epoch 5/500
159/159 [==============================] - 1s 3ms/step - loss: 97.7576 - root_mean_squared_error: 9.8871 - val_loss: 78.4461 - val_root_mean_squared_error: 8.8568
Epoch 6/500
159/159 [==============================] - 1s 3ms/step - loss: 66.17

In [17]:
e2_pred_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 2)]               0         
                                                                 
 batch_normalization_1 (Batc  (None, 2)                8         
 hNormalization)                                                 
                                                                 
 dense_variational_3 (DenseV  (None, 10)               495       
 ariational)                                                     
                                                                 
 dense_variational_4 (DenseV  (None, 10)               6215      
 ariational)                                                     
                                                                 
 dense_variational_5 (DenseV  (None, 1)                77        
 ariational)                                               

In [44]:
_, rmse_e2_pred = e2_pred_model.evaluate(x_train, y_train, verbose=0)
print(f"Test RMSE: {rmse_e2_pred}")

Test RMSE: 1.204264760017395


In [19]:
ade1 = gdrdata.drop(['Z','El','Eta','E1[MeV]','E2[MeV]','W1[MeV]','W2[MeV]'],axis = 1).to_numpy()
ade1

array([[3.50000000e+01, 2.00000000e-01],
       [3.60000000e+01, 2.22222222e-01],
       [3.70000000e+01, 2.43243243e-01],
       ...,
       [3.37000000e+02, 3.47181009e-01],
       [3.38000000e+02, 3.49112426e-01],
       [3.39000000e+02, 3.51032448e-01]])

In [20]:
y1 = gdrdata['E1[MeV]'].to_numpy()
y1

array([21.27, 18.63, 17.55, ..., 10.4 , 10.38,  8.3 ])

In [21]:
x1_train, x1_test, y1_train, y1_test = train_test_split(ade1, y1,train_size = 0.85)
print(len(x1_train))

5088


In [22]:
num_epochs = 500
mse_loss = keras.losses.MeanSquaredError()
e1_pred_model = create_bnn_model(int(x1_train.size ))
run_experiment(e1_pred_model, mse_loss, x1_train,y1_train,x1_test,y1_test)

Start training the model...
Epoch 1/500
159/159 [==============================] - 5s 11ms/step - loss: 159.6706 - root_mean_squared_error: 12.6360 - val_loss: 154.5984 - val_root_mean_squared_error: 12.4337
Epoch 2/500
159/159 [==============================] - 1s 3ms/step - loss: 138.0486 - root_mean_squared_error: 11.7493 - val_loss: 113.7586 - val_root_mean_squared_error: 10.6657
Epoch 3/500
159/159 [==============================] - 1s 3ms/step - loss: 110.6541 - root_mean_squared_error: 10.5191 - val_loss: 109.1211 - val_root_mean_squared_error: 10.4460
Epoch 4/500
159/159 [==============================] - 1s 6ms/step - loss: 85.0759 - root_mean_squared_error: 9.2235 - val_loss: 87.9741 - val_root_mean_squared_error: 9.3793
Epoch 5/500
159/159 [==============================] - 2s 11ms/step - loss: 63.3252 - root_mean_squared_error: 7.9576 - val_loss: 50.5847 - val_root_mean_squared_error: 7.1121
Epoch 6/500
159/159 [==============================] - 1s 5ms/step - loss: 41.0904 

In [23]:
_, rmse_e1_pred = e1_pred_model.evaluate(x1_train, y1_train, verbose=0)
print(f"Test RMSE: {rmse_e1_pred}")

Test RMSE: 1.496177315711975


In [28]:
adw1 = gdrdata.drop(['Z','El','Eta','E1[MeV]','E2[MeV]','W1[MeV]','W2[MeV]'],axis = 1).to_numpy()
adw1

array([[3.50000000e+01, 2.00000000e-01],
       [3.60000000e+01, 2.22222222e-01],
       [3.70000000e+01, 2.43243243e-01],
       ...,
       [3.37000000e+02, 3.47181009e-01],
       [3.38000000e+02, 3.49112426e-01],
       [3.39000000e+02, 3.51032448e-01]])

In [29]:
y2 = gdrdata['W1[MeV]'].to_numpy()
y2

array([7.76, 5.41, 4.95, ..., 2.35, 2.35, 1.84])

In [39]:
rmse_w1_pred = 0.99846115648948
rmse_w1_pred

0.99846115648948

In [30]:
x2_train, x2_test, y2_train, y2_test = train_test_split(adw1, y2,train_size = 0.85)
print(len(x2_train))

5088


In [31]:
num_epochs = 500
mse_loss = keras.losses.MeanSquaredError()
w1_pred_model = create_bnn_model(int(x1_train.size ))
run_experiment(w1_pred_model, mse_loss, x2_train,y2_train,x2_test,y2_test)

Start training the model...
Epoch 1/500
159/159 [==============================] - 5s 14ms/step - loss: 14.4308 - root_mean_squared_error: 3.7986 - val_loss: 11.7288 - val_root_mean_squared_error: 3.4245
Epoch 2/500
159/159 [==============================] - 1s 7ms/step - loss: 8.8578 - root_mean_squared_error: 2.9759 - val_loss: 6.0649 - val_root_mean_squared_error: 2.4624
Epoch 3/500
159/159 [==============================] - 1s 6ms/step - loss: 5.8169 - root_mean_squared_error: 2.4115 - val_loss: 5.6304 - val_root_mean_squared_error: 2.3725
Epoch 4/500
159/159 [==============================] - 1s 5ms/step - loss: 4.6001 - root_mean_squared_error: 2.1443 - val_loss: 3.8569 - val_root_mean_squared_error: 1.9634
Epoch 5/500
159/159 [==============================] - 1s 3ms/step - loss: 3.8100 - root_mean_squared_error: 1.9514 - val_loss: 2.9612 - val_root_mean_squared_error: 1.7202
Epoch 6/500
159/159 [==============================] - 1s 3ms/step - loss: 3.1571 - root_mean_squared_er

In [32]:
_, rmse_w1_pred = w1_pred_model.evaluate(x2_train, y2_train, verbose=0)
print(f"Test RMSE: {rmse_w1_pred}")

Test RMSE: 1.3072739839553833


In [33]:
adw2 = gdrdata.drop(['Z','El','Eta','E1[MeV]','E2[MeV]','W1[MeV]','W2[MeV]'],axis = 1).to_numpy()
adw2

array([[3.50000000e+01, 2.00000000e-01],
       [3.60000000e+01, 2.22222222e-01],
       [3.70000000e+01, 2.43243243e-01],
       ...,
       [3.37000000e+02, 3.47181009e-01],
       [3.38000000e+02, 3.49112426e-01],
       [3.39000000e+02, 3.51032448e-01]])

In [34]:
y3 = gdrdata['W2[MeV]'].to_numpy()
y3

array([7.76, 9.55, 9.49, ..., 3.38, 3.37, 3.25])

In [35]:
x3_train, x3_test, y3_train, y3_test = train_test_split(adw2, y3,train_size = 0.85)
print(len(x3_train))

5088


In [36]:
num_epochs = 500
mse_loss = keras.losses.MeanSquaredError()
w2_pred_model = create_bnn_model(int(x3_train.size ))
run_experiment(w2_pred_model, mse_loss, x3_train,y3_train,x3_test,y3_test)

Start training the model...
Epoch 1/500
159/159 [==============================] - 5s 13ms/step - loss: 26.0489 - root_mean_squared_error: 5.1036 - val_loss: 22.8704 - val_root_mean_squared_error: 4.7821
Epoch 2/500
159/159 [==============================] - 1s 6ms/step - loss: 17.8731 - root_mean_squared_error: 4.2275 - val_loss: 14.3656 - val_root_mean_squared_error: 3.7900
Epoch 3/500
159/159 [==============================] - 1s 5ms/step - loss: 11.3558 - root_mean_squared_error: 3.3696 - val_loss: 10.1060 - val_root_mean_squared_error: 3.1787
Epoch 4/500
159/159 [==============================] - 1s 4ms/step - loss: 7.4002 - root_mean_squared_error: 2.7200 - val_loss: 6.0049 - val_root_mean_squared_error: 2.4500
Epoch 5/500
159/159 [==============================] - 1s 3ms/step - loss: 4.5362 - root_mean_squared_error: 2.1293 - val_loss: 3.9592 - val_root_mean_squared_error: 1.9892
Epoch 6/500
159/159 [==============================] - 1s 3ms/step - loss: 3.8873 - root_mean_square

In [37]:
_, rmse_w2_pred = w2_pred_model.evaluate(x3_train, y3_train, verbose=0)
print(f"Test RMSE: {rmse_w2_pred}")

Test RMSE: 1.2093405723571777


# Comparision Before and after finetuning with BNN of given data

e1 rmse vs e1 prediction rmse

In [41]:
print("The deviation is reduced from", sigma_e1,"to", rmse_e1_pred,"after fine tuning with BNN network")

The deviation is reduced from 2.2000004234459576 to 1.496177315711975 after fine tuning with BNN network


e2 rmse vs e2 prediction rmse

In [45]:
print("The deviation is reduced from", sigma_e2,"to", rmse_e2_pred,"after fine tuning with BNN network")

The deviation is reduced from 2.1392005714053113 to 1.204264760017395 after fine tuning with BNN network


w1 rmse vs w1 prediction rmse

In [47]:
print("The deviation is reduced from", sigma_w1,"to", rmse_w1_pred,"after fine tuning with BNN network")

The deviation is reduced from 1.0946500160207928 to 0.99846115648948 after fine tuning with BNN network


w2 rmse vs w2 prediction rmse

In [46]:
print("The deviation is reduced from", sigma_w2,"to", rmse_w2_pred,"after fine tuning with BNN network")

The deviation is reduced from 1.2628291865343948 to 1.2093405723571777 after fine tuning with BNN network
